In [1]:
# Import dependencies
import pandas as pd

In [3]:
df = pd.read_csv('data_prepro/allStats.csv', index_col=0)

In [4]:
len(df)

68749

In [5]:
# Clean defense names in historical data
# Create function to check if last three characters are uppercase. Strip only the uppercase
def cleanDefNames(text):
    if text[-3:].isupper():
        text = text[:-3]
    else:
        text = text[:-2]
    return(text)
# Apply the function to the defenses
df.loc[df.pos=='DF', 'player'] = df.loc[df.pos=='DF', 'player'].apply(cleanDefNames)

# Rename past teams to their most recent name to account for name changes or city moves
df.team.replace(to_replace=['STL', 'LA'], value='LAR', inplace=True)
df.player.replace(to_replace=['St. Louis Rams'], value='Los Angeles Rams', inplace=True)
df.team.replace(to_replace=['OAK', 'LV'], value='LVR', inplace=True)
df.player.replace(to_replace=['Oakland Raiders'], value='Las Vegas Raiders', inplace=True)
df.team.replace(to_replace=['NE'], value='NEP', inplace=True)
df.team.replace(to_replace=['SD'], value='LAC', inplace=True)
df.team.replace(to_replace=['TB'], value='TBB', inplace=True)
df.team.replace(to_replace=['NO'], value='NOS', inplace=True)
df.team.replace(to_replace=['GB'], value='GBP', inplace=True)
df.team.replace(to_replace=['KC'], value='KCC', inplace=True)
df.team.replace(to_replace=['SF'], value='SFO', inplace=True)
df.team.replace(to_replace=['TB'], value='TBB', inplace=True)

df.opponent.replace(to_replace=['STL', 'LA'], value='LAR', inplace=True)
df.opponent.replace(to_replace=['OAK', 'LV'], value='LVR', inplace=True)
df.opponent.replace(to_replace=['NE'], value='NEP', inplace=True)
df.opponent.replace(to_replace=['SD'], value='LAC', inplace=True)
df.opponent.replace(to_replace=['TB'], value='TBB', inplace=True)
df.opponent.replace(to_replace=['NO'], value='NOS', inplace=True)
df.opponent.replace(to_replace=['GB'], value='GBP', inplace=True)
df.opponent.replace(to_replace=['KC'], value='KCC', inplace=True)
df.opponent.replace(to_replace=['SF'], value='SFO', inplace=True)
df.opponent.replace(to_replace=['TB'], value='TBB', inplace=True)

In [6]:
len(df)

68749

In [7]:
# Set NaN values to zero
df.fillna(0, inplace=True)

# Handle duplicates
dupList = df.loc[df['pos']!='DF'].groupby(['player', 'season', 'week'])['player'].count().sort_values(ascending=False)
dupList = dupList.loc[dupList>1].to_frame().index.get_level_values(0).drop_duplicates()
# Handle 2022 duplicates by appending position to player name
for p in dupList:
    df.loc[df.player==p, 'player'] = df.loc[df.player==p, 'player'] + df.loc[df.player==p, 'pos']


In [8]:
len(df)

68749

In [9]:
# Create summary df for current season prior to current week
currentSeason_df = df.copy()
# Create empty list
data=[]

# Loop through df
for i in range(len(currentSeason_df)):
    season = currentSeason_df.iloc[i]['season']
    player = currentSeason_df.iloc[i]['player']
    week = currentSeason_df.iloc[i]['week']
# filter for player where week < current week
    prevWeeks = currentSeason_df.loc[(currentSeason_df['season']==season) & (currentSeason_df['player']==player) & (currentSeason_df['week']<week)]
    ind = [season, week, player]
    prevComplete = []
    if len(prevWeeks) > 0:
     # take average
        guts = prevWeeks[['passA', 'passC', 'passY', 'passT', 'passI', 'pass2', 'rushA', 'rushY',
        'rushT', 'rush2', 'recC', 'recY', 'recT', 'rec2', 'fum', 'XPA', 'XPM',
        'FGA', 'FGM', 'FG50', 'defSack', 'defI', 'defSaf', 'defFum', 'defBlk',
        'defT', 'defPtsAgainst', 'defPassYAgainst', 'defRushYAgainst',
        'defYdsAgainst']].mean().to_list()
        
    else:
        guts = [0,0,0,0,0,
        0,0,0,0,0,
        0,0,0,0,0,
        0,0,0,0,0,
        0,0,0,0,0,
        0,0,0,0,0]
    ind.extend(guts)
    # # groupby and count games played
    gaPlayed = [len(prevWeeks)]
    ind.extend(gaPlayed)
    data.append(ind)
# Errors if no weeks < week
seasonCurrent = pd.DataFrame(data, columns=['season', 'week', 'player',
       'passA', 'passC', 'passY', 'passT', 'passI', 'pass2', 'rushA', 'rushY',
       'rushT', 'rush2', 'recC', 'recY', 'recT', 'rec2', 'fum', 'XPA', 'XPM',
       'FGA', 'FGM', 'FG50', 'defSack', 'defI', 'defSaf', 'defFum', 'defBlk',
       'defT', 'defPtsAgainst', 'defPassYAgainst', 'defRushYAgainst',
       'defYdsAgainst', 'gamesPlayed'])

# Rename all columns in seasonCurrent
colList = [(x + "_curr") for x in list(seasonCurrent.columns)]
seasonCurrent.columns = colList
seasonCurrent = seasonCurrent.rename(columns={'season_curr':'season',
 'week_curr':'week',
 'player_curr':'player'})


In [10]:
len(seasonCurrent)

68749

In [11]:

#Create summary dfs for each season
seasonsPrior1_df = df.copy()
seasonsPrior1_df['season'] = seasonsPrior1_df['season'] + 1
seasonsPrior1 = seasonsPrior1_df.groupby(['season', 'player']).mean()
seasonsPrior1['gamesPlayed'] = seasonsPrior1_df.groupby(['season', 'player'])['week'].count()
seasonsPrior1.drop(columns='week', inplace=True)
gamesPlayedCol = seasonsPrior1.pop('gamesPlayed')
seasonsPrior1.insert(0, 'gamesPlayed', gamesPlayedCol)
# Rename all columns in seasonsPrior1
colList = [(x + "_prior1") for x in list(seasonsPrior1.columns)]
seasonsPrior1.columns = colList
seasonsPrior1

# Create summary dfs for combined two seasons before
seasonsPrior2_df = df.copy()
seasonsPrior2_df['season'] = seasonsPrior2_df['season'] + 2
seasonsPrior2_df = pd.concat([seasonsPrior1_df, seasonsPrior2_df], axis=0, ignore_index=True)
seasonsPrior2 = seasonsPrior2_df.groupby(['season', 'player']).mean()
seasonsPrior2['gamesPlayed'] = seasonsPrior2_df.groupby(['season', 'player'])['week'].count()
seasonsPrior2.drop(columns='week', inplace=True)
gamesPlayedCol = seasonsPrior2.pop('gamesPlayed')
seasonsPrior2.insert(0, 'gamesPlayed', gamesPlayedCol)
# Rename all columns in seasonsPrior2
colList = [(x + "_prior2") for x in list(seasonsPrior2.columns)]
seasonsPrior2.columns = colList
seasonsPrior2

gamesPlayed_prior2  passA_prior2  passC_prior2  \
season player                                                            
2011.0 Aaron Brown                       3      0.000000      0.000000   
       Aaron Hernandez                  13      0.000000      0.000000   
       Aaron Pettrey                     2      0.000000      0.000000   
       Aaron Rodgers                    15     31.666667     20.800000   
       Adam Vinatieri                   16      0.000000      0.000000   
...                                    ...           ...           ...   
2023.0 Zach Pascal                      14      0.000000      0.000000   
       Zach Wilson                      12     30.250000     17.166667   
       Zack Moss                        12      0.000000      0.000000   
       Zane Gonzalez                    12      0.000000      0.000000   
       Zay Jones                        14      0.000000      0.000000   

                        passY_prior2  passT_prior2  passI_prior2  \
season player                                                      
2011.0 Aaron Brown          0.000000      0.000000      0.000000   
       Aaron Hernandez      0.000000      0.000000      0.000000   
       Aaron Pettrey        0.000000      0.000000      0.000000   
       Aaron Rodgers      261.466667      1.866667      0.733333   
       Adam Vinatieri       0.000000      0.000000      0.000000   
...                              ...           ...           ...   
2023.0 Zach Pascal          0.000000      0.000000      0.000000   
       Zach Wilson        187.250000      0.666667      0.916667   
       Zack Moss            0.000000      0.000000      0.000000   
       Zane Gonzalez        0.000000      0.000000      0.000000   
       Zay Jones            0.000000      0.000000      0.000000   

                        pass2_prior2  rushA_prior2  rushY_prior2  \
season player                                                      
2011.0 Aaron Brown          0.000000      5.666667     19.333333   
       Aaron Hernandez      0.000000      0.230769      3.615385   
       Aaron Pettrey        0.000000      0.000000      0.000000   
       Aaron Rodgers        0.000000      4.266667     23.733333   
       Adam Vinatieri       0.000000      0.000000      0.000000   
...                              ...           ...           ...   
2023.0 Zach Pascal          0.000000      0.142857      1.500000   
       Zach Wilson          0.083333      2.250000     13.416667   
       Zack Moss            0.000000      7.583333     28.083333   
       Zane Gonzalez        0.000000      0.000000      0.000000   
       Zay Jones            0.000000      0.142857      0.214286   

                        rushT_prior2  ...  defSack_prior2  defI_prior2  \
season player                         ...                                
2011.0 Aaron Brown          0.000000  ...             0.0          0.0   
       Aaron Hernandez      0.000000  ...             0.0          0.0   
       Aaron Pettrey        0.000000  ...             0.0          0.0   
       Aaron Rodgers        0.266667  ...             0.0          0.0   
       Adam Vinatieri       0.000000  ...             0.0          0.0   
...                              ...  ...             ...          ...   
2023.0 Zach Pascal          0.000000  ...             0.0          0.0   
       Zach Wilson          0.333333  ...             0.0          0.0   
       Zack Moss            0.333333  ...             0.0          0.0   
       Zane Gonzalez        0.000000  ...             0.0          0.0   
       Zay Jones            0.000000  ...             0.0          0.0   

                        defSaf_prior2  defFum_prior2  defBlk_prior2  \
season player                                                         
2011.0 Aaron Brown                0.0            0.0            0.0   
       Aaron Hernandez            0.0            0.0            0.0   
       Aaron Pettrey              0.0       

In [12]:
print(len(seasonsPrior1))
print(len(seasonsPrior2))

7432
9796


In [23]:
# Merge df, currentSeason, seasonsPrior1, and seasonsPrior1
final = df.copy()

# Merge df, currentSeason, seasonsPrior1, and seasonsPrior1
final = final.merge(
    seasonCurrent, how='left', on=['season', 'week', 'player']
    ).merge(
        seasonsPrior1, how='left', on=['season', 'player']
        ).merge(
            seasonsPrior2, how='left', on=['season', 'player']
            )
# Remove rows where we do not have two full years prior data
final = final.loc[final['season']>2011]
# fill NA values for players who do not have two years prior data
final.fillna(0, inplace=True)

In [24]:
print(len(final))
print(len(final.groupby(['player', 'season'])))


57483
6242


In [25]:
# select only defenses
allDef = final.loc[final['pos']=='DF']

# Rename all columns in allDef
colList = [(x + "_opp") for x in list(allDef.columns)]
allDef.columns = colList
allDef = allDef.rename(columns={
       'season_opp':'season',
       'week_opp':'week',
       'team_opp':'team',
       'player_opp':'player'})

allDef = allDef[['season', 'week', 'team', 
       'defSack_curr_opp', 'defI_curr_opp',
       'defSaf_curr_opp', 'defFum_curr_opp', 'defBlk_curr_opp',
       'defT_curr_opp', 'defPtsAgainst_curr_opp', 'defPassYAgainst_curr_opp',
       'defRushYAgainst_curr_opp', 'defYdsAgainst_curr_opp',
       'defSack_prior1_opp', 'defI_prior1_opp', 'defSaf_prior1_opp',
       'defFum_prior1_opp', 'defBlk_prior1_opp', 'defT_prior1_opp',
       'defPtsAgainst_prior1_opp', 'defPassYAgainst_prior1_opp',
       'defRushYAgainst_prior1_opp', 'defYdsAgainst_prior1_opp',]]

# Connect opponents to defenses
final = final.merge(allDef, how='left', left_on=['season', 'week', 'opponent'], right_on=['season', 'week', 'team'],)
final.drop(columns=['team_y'], inplace=True)
final.rename(columns={'team_x':'team'}, inplace=True)

In [27]:
# Read ages
xlAge = pd.read_csv('data_raw/ages_historical.csv', index_col=0)
dfAge = xlAge[['Player', 'Season', 'Age']]
dfAge.columns = ['player', 'season', 'age']
dfAge = dfAge.drop_duplicates(['player', 'season'])

# merge ages into final df
final = final.merge(dfAge, how='left', on=['season', 'player'])
ageCol = final.pop('age')
final.insert(4, 'age', ageCol)

# Change defense ages to 0
final.loc[final['pos']=='DF', 'age'] = 0
# Set duplicate player ages to 25
final.loc[final.age.isna(), 'age'] = 25

In [29]:
final = final.dropna()
len(final)

57483

In [30]:
# Account for the way WRS are set in slot, left, and right
# Map into three-tiered system
final['posRank'] = final['posRank'].map({
    'QB1':'QB1', 
    'QB2':'QB2', 
    'QB3':'QB3', 
    'QB4':'QB3',
    'QB5':'QB3', 
    'RB1':'RB1', 
    'RB2':'RB2', 
    'RB3':'RB3', 
    'RB4':'RB3', 
    'RB5':'RB3',
    'RB6':'RB3', 
    'RB7':'RB3', 
    'WR1': 'WR1', 
    'WR2': 'WR1', 
    'WR3': 'WR1', 
    'WR4': 'WR2', 
    'WR5': 'WR2', 
    'WR6': 'WR2', 
    'WR7': 'WR3', 
    'WR8': 'WR3',
    'TE1':'TE1', 
    'TE2':'TE2', 
    'TE3':'TE3', 
    'TE4':'TE3', 
    'TE5':'TE3', 
    'PK1':'PK1', 
    'PK2':'PK2', 
    'PK3':'PK3',
    'DF1':'DF1'
    })


In [32]:
final.to_csv('data_cleaned/modelSource.csv')